In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import numpy as np
from astropy import units
import os
import astropy.units as u
import matplotlib.pyplot as plt

import snapshot_obj
import dataset_compute

import importlib

In [ ]:
importlib.reload(snapshot_obj)
importlib.reload(dataset_compute)

# Radial distribution of satellites in the Local Group

Count the accumulation of satellites with radius from the centres of potential of the MW and M31 galaxies **together**.

Create a dictionary of the datasets from each simulation. 

In [ ]:
snap_id = 127
sim_ids = ["V1_MR_fix", "V1_MR_curvaton_p082_fix"]
names = ["plain-LCDM", "curv-p082"]

# Define M31 and MW in each simulation:
m31 = [(1,0), (1,0)]
mw = [(2,0), (1,1)]

data = {"By_r" : {}, "By_GN" : {}}
for name, sim_id, m31_ns, mw_ns in zip(names, sim_ids, m31, mw):
    for entry in data.values():
        entry[name] = {"snapshot": snapshot_obj.Snapshot(sim_id, snap_id, name=name),
                      "M31_identifier": m31_ns,
                      "MW_identifier": mw_ns}

## Get data for plotting

Read the datasets from each simulation, convert the units and split into satellites/isolated and luminous/dark:

In [ ]:
distinction = "By_r"

for key, sim_data in data[distinction].items():    
    # Get data:
    snap = sim_data["snapshot"]
    cops = snap.get_subhalos("CentreOfPotential")
    max_point = snap.get_subhalos("Max_Vcirc", "Extended")
    vmax = max_point[:,0] * units.cm.to(units.km)
    
    # Compute masking arrays:
    mask_lum, mask_dark = dataset_compute.split_luminous(snap)
    mask_nonzero_vmax = dataset_compute.prune_vmax(snap)
    if distinction == "By_r":
        mask_sat, mask_isol = dataset_compute.split_satellites_by_distance(
            snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
    elif distinction == "By_GN":
        mask_sat, mask_isol = dataset_compute.split_satellites_by_group_number(
            snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
        
    # In case M31 and MW are identified as the same halo, remove intersection:
    mask_sat[1] = np.logical_and(mask_sat[1], np.logical_not(mask_sat[0]))
    
    # Compute distances to M31:
    m31_cop = cops[snap.index_of_halo(sim_data["M31_identifier"][0],
                                      sim_data["M31_identifier"][1])]
    dist_to_m31 = dataset_compute.distance_to_point(snap, m31_cop) \
                    * units.cm.to(units.kpc)
    
    # Sort by distance and add a dummy point far away 
    # (to continue the curves to the right):
    sort_idx = np.argsort(dist_to_m31)
    sim_data["R_M31"] = np.concatenate([dist_to_m31[sort_idx], [10000]])
    sim_data["Vmax_M31"] = np.concatenate([vmax[sort_idx], [0.01]])
       
    # Add these selections to the data dict:
    sim_data["Selections"] = {
        "M31" : {
            "Luminous" : 
            np.concatenate([
                np.logical_and.reduce([mask_sat[0], mask_lum, mask_nonzero_vmax])[sort_idx],
                [True]
            ]),
            "Dark" : 
            np.concatenate([
                np.logical_and.reduce([mask_sat[0], mask_dark, mask_nonzero_vmax])[sort_idx],
                [True]
            ])
        }
    }
    
    # Compute distances to MW:
    mw_cop = cops[snap.index_of_halo(sim_data["MW_identifier"][0],
                                     sim_data["MW_identifier"][1])]
    dist_to_mw = dataset_compute.distance_to_point(snap, mw_cop) \
                    * units.cm.to(units.kpc)
    
    # Sort by distance and add a dummy point far away 
    # (to continue the curves to the right):
    sort_idx = np.argsort(dist_to_mw)
    sim_data["R_MW"] = np.concatenate([dist_to_mw[sort_idx], [10000]])
    sim_data["Vmax_MW"] = np.concatenate([vmax[sort_idx], [0.01]])
       
    # Add these selections to the data dict:
    sim_data["Selections"].update({
        "MW" : {
            "Luminous" : 
            np.concatenate([
                np.logical_and.reduce([mask_sat[1], mask_lum, mask_nonzero_vmax])[sort_idx],
                [True]
            ]),
            "Dark" : 
            np.concatenate([
                np.logical_and.reduce([mask_sat[1], mask_dark, mask_nonzero_vmax])[sort_idx],
                [True]
            ])
        }
    })

        
    # Compute distances to LG centre:
    lg_centre = dataset_compute.compute_LG_centre(
        snap, sim_data["M31_identifier"], sim_data["MW_identifier"]
    )
    dist_to_lg = dataset_compute.distance_to_point(snap, lg_centre) \
                    * units.cm.to(units.kpc)

    # Sort by distance and add a dummy point far away 
    # (to continue the curves to the right):
    sort_idx = np.argsort(dist_to_lg)
    sim_data["R_LG"] = np.concatenate([dist_to_lg[sort_idx], [10000]])
    sim_data["Vmax_LG"] = np.concatenate([vmax[sort_idx], [0.01]])
       
    # Add these selections to the data dict:
    sim_data["Selections"].update({
        "Isolated" : {
            "Luminous" : 
            np.concatenate([
                np.logical_and.reduce([mask_isol, mask_lum, mask_nonzero_vmax])[sort_idx],
                [True]
            ]),
            "Dark" : 
            np.concatenate([
                np.logical_and.reduce([mask_isol, mask_dark, mask_nonzero_vmax])[sort_idx],
                [True]
            ])
        }
    })

In [ ]:
distinction = "By_GN"

for key, sim_data in data[distinction].items():    
    # Get data:
    snap = sim_data["snapshot"]
    cops = snap.get_subhalos("CentreOfPotential")
    max_point = snap.get_subhalos("Max_Vcirc", "Extended")
    vmax = max_point[:,0] * units.cm.to(units.km)
    
    # Compute masking arrays:
    mask_lum, mask_dark = dataset_compute.split_luminous(snap)
    mask_nonzero_vmax = dataset_compute.prune_vmax(snap)
    if distinction == "By_R":
        mask_sat, mask_isol = dataset_compute.split_satellites_by_distance(
            snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
    elif distinction == "By_GN":
        mask_sat, mask_isol = dataset_compute.split_satellites_by_group_number(
            snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
        
    # In case M31 and MW are identified as the same halo, remove intersection:
    mask_sat[1] = np.logical_and(mask_sat[1], np.logical_not(mask_sat[0]))
    
    # Compute distances to M31:
    m31_cop = cops[snap.index_of_halo(sim_data["M31_identifier"][0],
                                      sim_data["M31_identifier"][1])]
    dist_to_m31 = dataset_compute.distance_to_point(snap, m31_cop) \
                    * units.cm.to(units.kpc)
    
    # Sort by distance and add a dummy point far away 
    # (to continue the curves to the right):
    sort_idx = np.argsort(dist_to_m31)
    sim_data["R_M31"] = np.concatenate([dist_to_m31[sort_idx], [10000]])
    sim_data["Vmax_M31"] = np.concatenate([vmax[sort_idx], [0.01]])
       
    # Add these selections to the data dict:
    sim_data["Selections"] = {
        "M31" : {
            "Luminous" : 
            np.concatenate([
                np.logical_and.reduce([mask_sat[0], mask_lum, mask_nonzero_vmax])[sort_idx],
                [True]
            ]),
            "Dark" : 
            np.concatenate([
                np.logical_and.reduce([mask_sat[0], mask_dark, mask_nonzero_vmax])[sort_idx],
                [True]
            ])
        }
    }
    
    # Compute distances to MW:
    mw_cop = cops[snap.index_of_halo(sim_data["MW_identifier"][0],
                                     sim_data["MW_identifier"][1])]
    dist_to_mw = dataset_compute.distance_to_point(snap, mw_cop) \
                    * units.cm.to(units.kpc)
    
    # Sort by distance and add a dummy point far away 
    # (to continue the curves to the right):
    sort_idx = np.argsort(dist_to_mw)
    sim_data["R_MW"] = np.concatenate([dist_to_mw[sort_idx], [10000]])
    sim_data["Vmax_MW"] = np.concatenate([vmax[sort_idx], [0.01]])
       
    # Add these selections to the data dict:
    sim_data["Selections"].update({
        "MW" : {
            "Luminous" : 
            np.concatenate([
                np.logical_and.reduce([mask_sat[1], mask_lum, mask_nonzero_vmax])[sort_idx],
                [True]
            ]),
            "Dark" : 
            np.concatenate([
                np.logical_and.reduce([mask_sat[1], mask_dark, mask_nonzero_vmax])[sort_idx],
                [True]
            ])
        }
    })

        
    # Compute distances to LG centre:
    lg_centre = dataset_compute.compute_LG_centre(
        snap, sim_data["M31_identifier"], sim_data["MW_identifier"]
    )
    dist_to_lg = dataset_compute.distance_to_point(snap, lg_centre) \
                    * units.cm.to(units.kpc)

    # Sort by distance and add a dummy point far away 
    # (to continue the curves to the right):
    sort_idx = np.argsort(dist_to_lg)
    sim_data["R_LG"] = np.concatenate([dist_to_lg[sort_idx], [10000]])
    sim_data["Vmax_LG"] = np.concatenate([vmax[sort_idx], [0.01]])
       
    # Add these selections to the data dict:
    sim_data["Selections"].update({
        "Isolated" : {
            "Luminous" : 
            np.concatenate([
                np.logical_and.reduce([mask_isol, mask_lum, mask_nonzero_vmax])[sort_idx],
                [True]
            ]),
            "Dark" : 
            np.concatenate([
                np.logical_and.reduce([mask_isol, mask_dark, mask_nonzero_vmax])[sort_idx],
                [True]
            ])
        }
    })

## Plot

In [ ]:
# Construct saving location:
filename = 'distinctions_r_distribution.png'
    
home = os.path.dirname(snapshot_obj.__file__)
path = os.path.join(home,"Figures", "MediumResolution")
filename = os.path.join(path, filename)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, sharex="col", sharey="row", figsize=(8,4))
plt.subplots_adjust(wspace=0.02)
plt.subplots_adjust(hspace=0.02)

y_down = 1; y_up = 3000
x_down_sat = 0; x_up_sat = 500
x_down_isol = 0; x_up_isol = 2000

# axes[0,0].yaxis.set_ticks_position("both")
# axes[1,0].yaxis.set_ticks_position("both")
# axes[0,1].yaxis.set_ticks_position("right")
# axes[0,2].yaxis.set_ticks_position("right")
# axes[1,1].yaxis.set_ticks_position("right")
# axes[1,2].yaxis.set_ticks_position("right")

for i, ax in enumerate(axes.flatten()):
    ax.tick_params(which="both", direction='in')
    
for i in range(np.size(axes, axis=0)):
    axes[i,0].set_yscale("log")
    axes[i,0].set_ylim(y_down, y_up)
    axes[i,0].set_ylabel('$N(>v_{\mathrm{max}})$')
    
    
axes[-1,0].set_xlabel("$r_\mathrm{M31}[\mathrm{kpc}]$")
axes[-1,1].set_xlabel("$r_\mathrm{MW}[\mathrm{kpc}]$")
axes[-1,2].set_xlabel("$r_\mathrm{LG}[\mathrm{kpc}]$")
for i in range(np.size(axes, axis=1)):
    if i < 2:
        axes[-1,i].set_xlim(x_down_sat, x_up_sat)
    else:
        axes[-1,i].set_xlim(x_down_isol, x_up_isol)

# axes[0,2].set_yticklabels([])
# axes[1,2].set_yticklabels([])

axes[0,0].set_title('Andromeda')
axes[0,1].set_title('Milky Way')
axes[0,2].set_title('Isolated')

# Construct marker size function:
smin = 0.1; smax = 20
masses = data["By_GN"]["plain-LCDM"]["Vmax_M31"]
masses = masses[data["By_GN"]["plain-LCDM"]["Selections"]["M31"]["Luminous"]]
mmax = np.min(masses)
mmin = np.max(masses)
def markers_func(x):
    return 1/(mmax-mmin) * (smin*(x-mmin) - smax*(x-mmax))

# Set colors:
color = ["black", "red", "blue", "green"]

offset = 0.03 # Offset for scatter points 
for j, distinction in enumerate(list(data.keys())):
    print(distinction)
    for i, (name, entry) in enumerate(data[distinction].items()):

        mask_lum = entry["Selections"]["M31"]["Luminous"]
        r = entry["R_M31"]
        vmax = entry["Vmax_M31"]
        cnt_lum = np.arange(1, np.sum(mask_lum) + 1) 
        axes[j,0].plot(r[mask_lum], cnt_lum, 
                     c=color[i], alpha=0.3, linestyle="solid",
                     label='{} luminous'.format(name))
        axes[j,0].scatter(r[mask_lum], cnt_lum * 10**offset, s=markers_func(vmax[mask_lum]),
                        facecolors="none",  edgecolors=color[i], alpha=0.7)

        mask_dark = entry["Selections"]["M31"]["Dark"]
        r = entry["R_M31"]
        vmax = entry["Vmax_M31"]
        cnt_dark = np.arange(1, np.sum(mask_dark) + 1) 
        axes[j,0].plot(r[mask_dark], cnt_dark, 
                     c=color[i], alpha=0.3, linestyle="dashed",
                     label='{} luminous'.format(name))
        axes[j,0].scatter(r[mask_dark], cnt_dark * 10**offset, s=markers_func(vmax[mask_dark]),
                        c=color[i], alpha=0.7)

        mask_lum = entry["Selections"]["MW"]["Luminous"]
        r = entry["R_MW"]
        vmax = entry["Vmax_MW"]
        cnt_lum = np.arange(1, np.sum(mask_lum) + 1) 
        axes[j,1].plot(r[mask_lum], cnt_lum, 
                     c=color[i], alpha=0.3, linestyle="solid",
                     label='{} luminous'.format(name))
        axes[j,1].scatter(r[mask_lum], cnt_lum * 10**offset, s=markers_func(vmax[mask_lum]),
                        facecolors="none",  edgecolors=color[i], alpha=0.7)

        mask_dark = entry["Selections"]["MW"]["Dark"]
        r = entry["R_MW"]
        vmax = entry["Vmax_MW"]
        cnt_dark = np.arange(1, np.sum(mask_dark) + 1) 
        axes[j,1].plot(r[mask_dark], cnt_dark, 
                     c=color[i], alpha=0.3, linestyle="dashed",
                     label='{} luminous'.format(name))
        axes[j,1].scatter(r[mask_dark], cnt_dark * 10**offset, s=markers_func(vmax[mask_dark]),
                        c=color[i], alpha=0.7)

        
        mask_lum = entry["Selections"]["Isolated"]["Luminous"]
        r = entry["R_LG"]
        vmax = entry["Vmax_LG"]
        cnt_lum = np.arange(1, np.sum(mask_lum) + 1) 
        axes[j,2].plot(r[mask_lum], cnt_lum, 
                     c=color[i], alpha=0.3, linestyle="solid",
                     label='{} luminous'.format(name))
        axes[j,2].scatter(r[mask_lum], cnt_lum * 10**offset, s=markers_func(vmax[mask_lum]),
                        facecolors="none",  edgecolors=color[i], alpha=0.7)

        mask_dark = entry["Selections"]["Isolated"]["Dark"]
        r = entry["R_LG"]
        vmax = entry["Vmax_LG"]
        cnt_dark = np.arange(1, np.sum(mask_dark) + 1) 
        axes[j,2].plot(r[mask_dark], cnt_dark, 
                     c=color[i], alpha=0.3, linestyle="dashed",
                     label='{} luminous'.format(name))
        axes[j,2].scatter(r[mask_dark], cnt_dark * 10**offset, s=markers_func(vmax[mask_dark]),
                        c=color[i], alpha=0.7)

plt.savefig(filename, dpi=200)